In [14]:
import pandas as pd
import json
import re

In [5]:
path = 'data/01.json'
with open(path, encoding="utf8") as file: 
    content = file.read()
parsed = json.loads(content)  
#pop first layer of the records
parsed.update(parsed.pop('records'))

In [6]:
df = pd.DataFrame.from_dict(parsed, orient='index')

### Subject

In [18]:
#Subject Access fields
subject_columns = df.filter(regex=("^6.*")).columns

In [19]:
#function to extract the name of a subject
def extract_name(text):
    r = re.search(r'(?<=\'a\':\s\')(.*?)(?=\')', text)
    return r.group(1) if r else None


df[subject_columns].apply(lambda x: extract_name(str(x)))

610    Deutsches Literaturarchiv Marbach
648                           Geschichte
689                                 None
600                    Burckhardt, Jacob
650                     Kunstbetrachtung
651                          Deutschland
655                          Festschrift
688       Jacob Burckhardt (1818 - 1897)
630                         Fitzcarraldo
653                     Palladio, Andrea
611                                 None
dtype: object

In [20]:
df

,001,003,005,007,008,035,040,041,049,084,...,800,534,515,730,026,850,538,740,533,535
BV000023649,BV000023649,DE-604,20150724,t,870612s1982 |||| 00||| ger d,{'a': '(DE-599)BVBBV000023649'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ger'},{'a': 'DE-Y7'},"{'a': 'GK 9067', '0': '(DE-625)41263:', '2': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000024214,BV000024214,DE-604,20100324,t,870612s1982 gw a||| |||| 00||| ger d,{'a': '(DE-599)BVBBV000024214'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ger'},{'a': 'DE-188'},"{'a': 'NB 6128', '0': '(DE-625)124171:', '2': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000024767,BV000024767,DE-604,20170322,t,870612s1982 gw |||| 00||| ger d,{'a': '(DE-599)BVBBV000024767'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ger'},{'a': 'DE-Y3'},"{'a': '5,1', '2': 'ssgn'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000025578,BV000025578,DE-604,20100903,t,870612s1982 b||| |||| 00||| ger d,{'a': '(DE-599)BVBBV000025578'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ger'},{'a': 'DE-Y2'},"{'a': 'GN 9999', '0': '(DE-625)43130:', '2': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000026429,BV000026429,DE-604,20210505,t,870612s1982 a||| |||| 01||| ita d,{'a': '(DE-599)BVBBV000026429'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},"{'a': 'LI 99999', '0': '(DE-625)97471:', '2': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV001554556,BV001554556,DE-604,20010720,t,890628s1646 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554556'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554561,BV001554561,DE-604,00000000000000.0,t,890628s1635 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554561'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554563,BV001554563,DE-604,20160826,t,890628s1816 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554563'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554686,BV001554686,DE-604,20160115,t,890628s1769 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554686'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-83'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### List of columns that we might be interested in
041: 'language_code'  
049: 'holdings'  
100: 'author_person'    
110: 'author_corporate'  
245: 'title'  
264: 'publication'  
300: 'physical_description'  
600: 'subject_personal'  
611: 'subject_corporate'  
648: 'subject_chronological'  
650: 'subject_topical'  
keep all the 6XX for topics  

